# ファインチューニング技術：SFT、DPO、RFTの選択（DPOガイドを含む）

*このガイドは、OpenAIのAPIについてある程度の経験があり、ファインチューニングされたモデルを研究やその他の適切な用途に使用したい開発者およびML実践者向けです。OpenAIのサービスは、医療状態の個別治療や診断を目的としたものではなく、当社の[適用される利用規約](https://openai.com/policies/)に従います。*

このガイドでは、OpenAIがサポートするファインチューニング手法について説明し、特に各手法が最適な用途と不適切な用途を強調することで、あなたのユースケースに最も適した技術を特定できるよう支援します。その後、特定の手法である直接選好最適化（DPO）について詳しく説明し、他の技術については既存のガイドへのリンクを提供します。

**ファインチューニングとは何か？** ファインチューニングは、より小さなドメイン固有のデータセットで継続的に訓練を行い、特定のタスクに対してモデルを最適化するプロセスです。通常、ファインチューニングを行う主な理由は2つあります：
1. 特定のタスクでのモデル性能を向上させる
2. モデルの効率性を向上させる（必要なトークン数を削減する、専門知識をより小さなモデルに蒸留するなど）

現在、OpenAIプラットフォームは4つのファインチューニング手法をサポートしています：
- **教師ありファインチューニング（SFT）：** この技術は、入力-出力ペアを使用した従来の教師あり学習を採用してモデルパラメータを調整します。訓練プロセスでは、提供された例全体で予測出力と目標出力の差を最小化するようにモデルの重みを調整します。モデルは提供されたペアで見つけた特徴を複製します。
- **ビジョンファインチューニング：** この技術は、統一された訓練フレームワークでテキストと画像の両方を処理することで、教師ありファインチューニングをマルチモーダルデータに拡張します。訓練プロセスでは、テキスト-画像ペア全体でエラーを最小化するようにモデルの重みを調整し、その結果、モデルの画像入力に対する理解を向上させます。
- **直接選好最適化（DPO）：** この技術は、ペアワイズ比較（例：好ましい応答例と拒否された応答例）を使用して、特定の出力を他の出力よりも優先するようにモデルを最適化します。モデルは提供された比較データで見つけた選好パターンを複製することを学習します。
- **強化ファインチューニング（RFT）：** この技術は、複雑な目標に対してモデルをファインチューニングするために、報酬信号（採点者や報酬モデルを通じて）を伴う強化学習を使用します。RFTでは、モデルは訓練中に与えられたプロンプトに対して出力を生成し、各出力の品質が評価されます。その後、モデルのパラメータは報酬を最大化するように更新され、より良い結果につながる行動を強化します。この反復的なフィードバックループにより、モデルは推論や意思決定戦略を改善するよう促されます。

適切なファインチューニング手法を選択するために、以下の表では各手法が最適なシナリオと、適していないシナリオをまとめています：

| **手法**                            | **適している用途**                                                                                                                                                                                                                                                                                    | **適していない用途**                                                                                            |
| ---------------------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ----------------------------------------------------------------------------------------------------------- |
| **教師ありファインチューニング (SFT)**         | モデルに既に存在する知識を強調する場合。<br>レスポンスの構造やトーンをカスタマイズする場合。<br>特定のフォーマットでコンテンツを生成する場合。<br>複雑な指示を教える、または指示に従う際の失敗を修正する場合。<br>コスト/レイテンシを最適化する場合（プロンプトからのトークン削減や蒸留）。 | 全く新しい知識を追加する場合（代わりにRAGを検討）。<br>主観的な品質を扱うタスク。                     |
| **ビジョンファインチューニング**                   | 専門的な視覚認識タスク（例：画像分類）。<br>ドメイン固有の画像理解。<br>複雑なプロンプトに対する指示実行の失敗を修正する場合。                                                                               | 純粋にテキストベースのタスク。<br>特定のコンテキストがない汎用的な視覚タスク。<br>一般的な画像理解。 |
| **直接選好最適化 (DPO)** | モデル出力を主観的な好み（トーン、丁寧さ）に合わせる場合。<br>人間による評価フィードバックを通じて出力を改善する場合。<br>微妙な行動アライメントを実現する場合。                                                                                                                                 | 完全に新しいタスクを学習する場合。<br>明確な人間の選好シグナルがないタスク。                             |
| **強化学習ファインチューニング (RFT)**      | 高度な推論を必要とする複雑なドメイン固有タスク。<br>既存の部分的な能力を改善する場合（創発的行動の促進）。<br>測定可能なフィードバックがあるタスク。<br>明示的なラベルが限られているが、報酬シグナルを定義できるシナリオ。                                                                     | モデルが初期スキルを持たないタスク。<br>明確なフィードバックや測定可能なシグナルがないタスク。          |

現在、以下の既存のCookbookが利用可能です：

- 教師ありファインチューニング（SFT）：(1) [How to fine-tune chat models](https://cookbook.openai.com/examples/how_to_finetune_chat_models) (2) [Leveraging model distillation to fine-tune a model](https://cookbook.openai.com/examples/leveraging_model_distillation_to_fine-tune_a_model)
- Visionファインチューニング：[Vision fine-tuning on GPT-4o for visual question answering](https://cookbook.openai.com/examples/multimodal/vision_fine_tuning_on_gpt4o_for_visual_question_answering)
- 強化ファインチューニング（RFT）：(1) [Reinforcement fine-tuning (RFT)](https://cookbook.openai.com/examples/reinforcement_fine_tuning)、(2) [Reinforcement fine-tuning for healthbench QA](https://cookbook.openai.com/examples/fine-tuned_qa/reinforcement_finetuning_healthbench)

直接選好最適化（DPO）については、このガイドで説明します。

## **Direct Preference Optimizationガイド**

上記で述べたように、[Direct Preference Optimization (DPO)](https://platform.openai.com/docs/guides/direct-preference-optimization)は、ペアワイズ選好データ（例：ランク付けされた応答のペア）を使用して言語モデルをファインチューニングするためのアライメント技術です。DPOは、通常は人間の選好からの明示的なペアワイズ比較を使用して、特定の出力を他の出力よりも優先するようにモデルを直接最適化します。このアプローチはアライメントを簡素化し、別個の報酬モデルや複雑な強化学習手順の必要性を排除するため、DPOは人間のフィードバックからの強化学習（RLHF）などの技術に対する軽量な代替手段となります。

DPOはいつ使用すべきでしょうか？DPOは、応答品質が主観的で客観的に測定できない場合や、トーン、スタイル、適切性、明確性などの微妙な基準が重要な場合、つまり通常複数の有効な出力が存在するケースで優れています。DPOがAI応答のアライメントに特に効果的なアプリケーションの例には以下があります：
- 会話型AI応答の向上
- コード生成品質とスタイルの改善
- 法的、倫理的、安全基準への準拠の確保
- ブランドボイス、プロフェッショナリズム、トーンの制御
- クリエイティブ出力とユーザーエクスペリエンスのカスタマイズ

好ましい完了と好ましくない完了の明示的なペアでファインチューニングすることで、DPOはモデル出力をこれらの微妙な選好にアライメントします。以下の表は、組織を代表する架空のAIアシスタントのペアワイズ選好データの例を示しており、好ましい応答は明確で、プロフェッショナルで、ブランド基準に沿ったものです。

| **質問例**                                           | **選択された応答**                                                                                                                                                                   | **拒否された応答**                                       |
|------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------|
| **Q1:** *あなたの製品をレビューするにはどうすればよいですか？*              | 製品レビューを送信するには、アカウントダッシュボードにアクセスし、製品を選択して「レビューを書く」をクリックしてください。正直な体験を共有し、主要機能を評価し、準備ができたら送信してください。        | よ、適当に星をつけるか何かしてよ、気楽にいこうぜ！         |
| **Q2:** *あなたの製品をレビューするにはどうすればよいですか？*              | フィードバックをお待ちしています！製品ページの「レビュー」セクションで「レビューを残す」をクリックし、評価を付けて、気に入った点や改善点についてコメントを追加してください。                   | 適当に何か書いて—正直、何でもいいよ。          |
| **Q3:** *この特定のエラーをトラブルシューティングするにはどうすればよいですか？* | エラー「X101」に対処するには、まずキャッシュをクリアし、次にインターネット接続を確認してください。問題が続く場合は、[サポート → トラブルシューティング → エラーX101]のステップバイステップガイドに従ってください。 | とりあえず再起動してみて。うまくいかなかったら、自分でなんとかして！ |

このガイドでは、ファインチューニングAPIを使用してDPOを適用する方法を説明します。あなたのユースケースに対して選好ファインチューニングジョブを成功させるために取るべき主要なステップを学びます。

以下の内容をカバーします：

- **1. 推奨ワークフロー**
- **2. デモンストレーションシナリオ**
- **3. データセットの生成**
- **4. ベースモデルのベンチマーク**
- **5. ファインチューニング**
- **6. ファインチューニングされたモデルの使用**

## **1. 推奨ワークフロー**
 
OpenAIでは以下のワークフローを推奨しています：
1. 好ましい応答のサブセットに対してSupervised Fine-Tuning（SFT）を実行する
2. SFTでファインチューニングされたモデルを開始点として、選好比較データを使用してDPOを適用する
 
Direct Preference Optimization（DPO）の前にSupervised Fine-Tuning（SFT）を実行することで、堅牢な初期ポリシーを確立し、モデルが既に正しい応答を好むようになるため、モデルのアライメントと全体的なパフォーマンスが向上します。これにより、DPO中の重みの更新幅が減少し、訓練が安定化され、DPOが微妙なニュアンスを効率的に調整できるようになることで過学習を防ぎます。その結果、SFT-then-DPOの組み合わせワークフローはより高速に収束し、より高品質な結果をもたらします。

このガイドでは、Direct Preference Optimization（DPO）の適用のみに焦点を当てます。ただし、使用ケースによっては、最初にSupervised Fine-Tuning（SFT）を実行することでパフォーマンスの向上が得られる場合があります。その場合は、上記にリンクされたSFTガイドに従い、結果として得られるモデルIDを保存して、それをDPOジョブの開始点として使用できます。

## **2. デモンストレーションシナリオ**

具体的に説明するために、架空のブランドの声とスタイルに従うよう顧客向けAIアシスタントをファインチューニングする手順を見ていきましょう。Good Vibes Corpという組織を想像してください。この組織は、親しみやすく熱意のあるトーンと個人的なタッチを誇りにしています。

彼らは、顧客向けAIアシスタントがこれらのブランドガイドライン（例：前向きな態度、丁寧な言葉遣い、親しみやすい締めくくり）を反映した方法で問い合わせに答えることを望んでおり、より一般的で素っ気ない回答よりもそのような応答を好みます。これはDPOに適したシナリオです：客観的に正しい回答形式は存在しませんが、好ましいスタイルは存在します。

DPOは、どのスタイルが好ましいかの比較からモデルが学習するのに役立ちます。以下の手順を概説します：(1) ペアになった応答（望ましいブランドボイスのものとそうでないもの）を持つプロンプトの合成選好データセットを生成する。(2) OpenAI evals APIを使用してベースモデルの性能を評価する。(3) 選好ファインチューニングに必要なJSONL形式でデータを準備してアップロードする。(4) OpenAI fine-tuning APIを使用してDPOでモデルをファインチューニングする。(5) OpenAI evals APIを使用してファインチューニング済みモデルを評価し、ブランドスタイルの選好がどのように改善されたかを示す。

このデモンストレーション用にデータセットを合成します。まず、より多くのバリエーションを生成するための質問のシードバンクを作成しましょう。

始めましょう！

In [1]:
! pip install openai nest-asyncio --quiet

In [1]:
PROMPT_SEED_POOL = [
    "Hi, I ordered a gadget last week. When will it arrive?",
    "Your product stopped working after two days. Can I get help?",
    "Do you offer discounts for long-term customers?",
    "Can I change the shipping address for my order?",
    "What is your return policy for damaged items?",
    "My tracking number hasn't updated in three days—can you check the status?",
    "How long is the warranty on your products, and how do I submit a claim?",
    "Can I add gift wrapping to my order before it ships?",
    "Do you accept PayPal or other alternative payment methods?",
    "Is there an option to expedite shipping if my order hasn't left the warehouse yet?",
]

## **3. データセットの生成**

次に、シードバンクから各プロンプトを取得し、関連する質問を生成する関数を定義します。まずこれらのプロンプトのバリエーションを生成し、次に各プロンプトに対して好ましい応答と拒否される応答の両方を生成することで、選好ペアのデータセットを作成します。

このデータセットは合成的なものであり、Direct Preference Optimizationの仕組みを説明することを目的としています。独自のアプリケーションを開発する際は、高品質な選好データセットを収集または精選する必要があります。注意：DPOに必要なデータ量は使用ケースによって異なりますが、一般的により多い方が良い（数千から数万）とされており、選好ペアについては順序ロジックが一貫している必要があります（例：A > B かつ B > C の場合、A > C となる）。

In [2]:
import asyncio
from openai import AsyncOpenAI
from typing import List, Dict, Any

async_client = AsyncOpenAI()

SYSTEM_PROMPT = "You are a customer-support assistant."


async def _generate_related_questions_from_prompt(
    prompt: str, k: int, sem: asyncio.Semaphore, *, model: str
) -> List[str]:
    """Return *k* distinct customer-service questions related to the given prompt."""
    out: List[str] = []
    async with sem:
        for _ in range(k):
            resp = await async_client.responses.create(
                model=model,
                input=[
                    {
                        "role": "system",
                        "content": (
                            "Return ONE distinct, realistic customer-service question "
                            "related in topic or theme to the following question, "
                            "but NOT a direct paraphrase."
                        ),
                    },
                    {"role": "user", "content": prompt},
                ],
                temperature=0.9,
                max_output_tokens=60,
            )
            out.append(resp.output_text.strip())
    return out


async def expand_prompt_pool(
    prompts: List[str], *, k: int = 3, concurrency: int = 32, model: str
) -> List[str]:
    """Expand each prompt into *k* related questions using the given model."""
    sem = asyncio.Semaphore(concurrency)
    tasks = [
        _generate_related_questions_from_prompt(p, k, sem, model=model) for p in prompts
    ]
    results = await asyncio.gather(*tasks)
    return [v for sub in results for v in sub]


async def _generate_preference_pair(
    prompt: str, sem: asyncio.Semaphore, *, model: str
) -> Dict[str, Any]:
    """Generate a preference pair for the given prompt."""
    async with sem:
        friendly_task = async_client.responses.create(
            model=model,
            input=[
                {
                    "role": "system",
                    "content": (
                        "You are Good Vibes Corp's exceptionally energetic, outrageously friendly and "
                        "enthusiastic support agent."
                    ),
                },
                {"role": "user", "content": prompt},
            ],
            temperature=0.7,  # higher temperature to increase creativity & on-brand tone adherence
            max_output_tokens=80,
        )
        blunt_task = async_client.responses.create(
            model=model,
            input=[
                {
                    "role": "system",
                    "content": "You are a terse, factual support agent with no empathy or politeness.",
                },
                {"role": "user", "content": prompt},
            ],
            temperature=0.3,  # lower temperature to limit creativity & emphasize tonal difference
            max_output_tokens=80,
        )
        friendly, blunt = await asyncio.gather(friendly_task, blunt_task)
        return {
            "input": {
                "messages": [
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": prompt},
                ]
            },
            "preferred_output": [
                {"role": "assistant", "content": friendly.output_text}
            ],
            "non_preferred_output": [
                {"role": "assistant", "content": blunt.output_text}
            ],
        }

これで、定義した関数を使用して、友好的な応答と無愛想な応答のペアを生成することでデータセットを構築します。友好的な応答は、ブランドが望むコミュニケーションスタイルを反映しています。効率性のために非同期で実行し、Direct Preference Optimizationに適したデータセットを作成します。

In [3]:
import math
import nest_asyncio


async def build_dataset(
    *,
    pair_count: int = 500,
    concurrency: int = 8,
    expand_prompt_pool_model: str,
    generate_preference_pair_model: str,
) -> List[Dict[str, Any]]:
    """Return *pair_count* preference pairs (single-shot expansion)."""

    seed = PROMPT_SEED_POOL
    deficit = max(0, pair_count - len(seed))
    k = max(1, math.ceil(deficit / len(seed)))

    expanded = await expand_prompt_pool(
        seed,
        k=k,
        concurrency=concurrency,
        model=expand_prompt_pool_model,
    )
    prompt_bank = (seed + expanded)[:pair_count]

    sem = asyncio.Semaphore(concurrency)
    tasks = [
        _generate_preference_pair(p, sem, model=generate_preference_pair_model)
        for p in prompt_bank
    ]
    return await asyncio.gather(*tasks)


nest_asyncio.apply()
pairs = await build_dataset(
    pair_count=500,
    concurrency=8,
    expand_prompt_pool_model="gpt-4.1-mini-2025-04-14",
    generate_preference_pair_model="gpt-4.1-mini-2025-04-14",
)
print(f"Dataset ready with {len(pairs)} pairs.")

Dataset ready with 500 pairs.


## **4. ベースモデルのベンチマーク**

以下では、データセットを訓練、検証、テストセットに分割します。また、訓練データセットからのサンプルを示します。これは、その入力ペアに対する好ましい応答（フレンドリーで、ブランドに合った）と好ましくない応答（ぶっきらぼうで、中立的な）の明確な違いを示しています。

In [4]:
# set dataset sizes
n = len(pairs)
n_train = int(0.8 * n)
n_val = int(0.1 * n)
n_test = n - n_train - n_val

# split dataset into train, test & validation
train_pairs = pairs[:n_train]
val_pairs = pairs[n_train : n_train + n_val]
test_pairs = pairs[n_train + n_val :]
train_pairs[0]

{'input': {'messages': [{'role': 'system',
    'content': 'You are a customer-support assistant.'},
   {'role': 'user',
    'content': 'Hi, I ordered a gadget last week. When will it arrive?'}]},
 'preferred_output': [{'role': 'assistant',
   'content': 'Hey there, awesome friend! 🌟 Thanks a bunch for reaching out! I’d LOVE to help you track down your gadget so you can start enjoying it ASAP! 🎉 Could you please share your order number or the email you used to place the order? Let’s make this delivery magic happen! 🚀✨'}],
 'non_preferred_output': [{'role': 'assistant',
   'content': 'Provide your order number for delivery status.'}]}

ファインチューニング前のモデルの性能を評価するために、自動採点者（LLM-as-a-Judge）を使用して、各回答の親しみやすさと共感性をスコア化します。採点者は各回答に0から4のスコアを割り当て、ベースモデルの平均ベースラインスコアを計算できるようにします。

これを行うために、まずテストセットでベースモデルの回答を生成し、次にOpenAI evals APIを使用して自動採点者による評価を作成・実行します。

In [5]:
async def generate_responses(
    testset,
    model,
    temperature=0.0,
    max_output_tokens=80,
    concurrency=8,
):
    """
    Generate responses for each prompt in the testset using the OpenAI responses API.
    Returns: List of dicts: [{"prompt": ..., "response": ...}, ...]
    """
    async_client = AsyncOpenAI()
    sem = asyncio.Semaphore(concurrency)

    async def get_response(prompt):
        async with sem:
            resp = await async_client.responses.create(
                model=model,
                input=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": prompt},
                ],
                temperature=temperature,
                max_output_tokens=max_output_tokens,
            )
            return {"prompt": prompt, "response": resp.output_text}

    tasks = [get_response(item["item"]["input"]) for item in testset]
    results = await asyncio.gather(*tasks)
    return results


# generate responses for the base model over the test set
base_model = "gpt-4.1-mini-2025-04-14"
testset = [
    {"item": {"input": pair["input"]["messages"][1]["content"]}} for pair in test_pairs
]
responses = await generate_responses(testset, model=base_model)

次に、OpenAI evals APIを使用して自動採点者による評価を作成・実行します。まず、LLM-as-a-Judgeのルーブリックを定義することから始めます。注意：データロギングを通じてレスポンスにアクセスするため、これが機能するには、データロギングが無効化されていない（zdr等を通じて）組織に所属している必要があります。これが自分の場合に当てはまるかわからない場合は、https://platform.openai.com/logs?api=responses にアクセスして、先ほど生成したレスポンスが確認できるかどうかを確認してください。

In [6]:
JUDGE_SYSTEM = """
You judge whether a reply matches Good Vibes Corp's desired tone:
energetic, super-friendly, enthusiastic.

Score 0-4 (higher = more energy):

4 - Highly enthusiastic: multiple upbeat phrases / emojis / exclamations, clear empathy, proactive help.
3 - Energetic & friendly: visible enthusiasm cue (≥1 emoji OR exclamation OR upbeat phrase), warm second-person tone.
2 - Pleasant: polite & positive but lacks obvious enthusiasm cues.
1 - Neutral: correct, businesslike, minimal warmth.
0 - Rude, negative, or unhelpful.
"""

In [7]:
from openai import OpenAI

sync_client = OpenAI()

# set judge model
judge_model = "gpt-4.1-2025-04-14"

# create the evaluation
logs_eval = sync_client.evals.create(
    name="Good Vibes Corp Tone Eval",
    data_source_config={
        "type": "logs",
    },
    testing_criteria=[
        {
            "type": "score_model",
            "name": "General Evaluator",
            "model": judge_model,
            "input": [
                {
                    "role": "system",
                    "content": JUDGE_SYSTEM,
                },
                {
                    "role": "user",
                    "content": (
                        "**User input**\n"
                        "{{item.input}}\n"
                        "**Response to evaluate**\n"
                        "{{sample.output_text}}"
                    ),
                },
            ],
            "range": [0, 4],
            "pass_threshold": 2,
        }
    ],
)

In [8]:
# run the evaluation
base_run = sync_client.evals.runs.create(
    name=base_model,
    eval_id=logs_eval.id,
    data_source={
        "type": "responses",
        "source": {"type": "responses", "limit": len(test_pairs)},
    },
)

In [10]:
# score base model
base_data = sync_client.evals.runs.output_items.list(
    eval_id=logs_eval.id, run_id=base_run.id
).data
base_scores = [s.results[0]["score"] for s in base_data]
print("Average score:", sum(base_scores) / len(base_scores))

Average score: 2.525


## **5. ファインチューニング**

ベースラインが確立されたので、トレーニングセットとDPOを使用してモデルをファインチューニングできます。このプロセスにより、先ほど作成した選好ペアに基づいて、モデルが望ましいスタイルに合致する応答を好むように学習させます。

注意：**beta (β)** は、Direct Preference Optimization (DPO)特有のファインチューニングハイパーパラメータです。これは0から2の範囲の浮動小数点数で、モデルの既存の動作を保持することと、新しい選好に合わせた応答に適応することのバランスを制御します。
- 高いβ（2に近い）：モデルをより保守的にし、以前の動作を強く優先します。ファインチューニングされたモデルは元のスタイルや特性からの逸脱を最小限に抑え、一貫性を重視し、急激な変化を避けます。
- 中程度のβ（1前後）：以前の動作への固執と新しい選好への適応のバランスを取ります。ほとんどの実用的なシナリオにおいて、合理的な出発点として推奨されます。
- 低いβ（0に近い）：積極的な適応を促し、モデルが新しく提供された選好をより顕著に優先するようになります。これにより大幅なスタイルの変化と明示的な選好との整合性の向上が得られる可能性がありますが、予期しない出力や過度に特化した出力につながる可能性があります。

技術的には、betaはDPO損失における対数確率の差をスケールします。βが大きいほど、シグモイドベースの損失関数がより小さな確率差で飽和し、重みの更新が小さくなります（したがって古い動作が保持されます）。特定のユースケースと安定性と適応性の間の望ましいトレードオフに合わせて最適な結果を得るために、β値を体系的に実験することが推奨されます。

In [11]:
import io
import json

# create training file
train_buf = io.BytesIO("\n".join(json.dumps(p) for p in train_pairs).encode())
train_buf.name = "train.jsonl"
train_file_id = sync_client.files.create(file=train_buf, purpose="fine-tune").id

# create validation file
val_buf = io.BytesIO("\n".join(json.dumps(p) for p in val_pairs).encode())
val_buf.name = "val.jsonl"
val_file_id = sync_client.files.create(file=val_buf, purpose="fine-tune").id

# create a fine-tuning job
ft = sync_client.fine_tuning.jobs.create(
    model=base_model,
    training_file=train_file_id,
    validation_file=val_file_id,
    method={
        "type": "dpo",
        "dpo": {
            "hyperparameters": {
                "n_epochs": 2,
                "beta": 0.1,
                "batch_size": 8,
            }
        },
    },
)
print(f"Fine-tuning job created: job_id = {ft.id}")

Fine-tuning job created: job_id = ftjob-5QPmA36QezFRGoXjuvIAPuAQ


## **6. ファインチューニングしたモデルの使用**

ファインチューニングが完了したら、同じテストセットでDPOチューニングしたモデルを評価します。ファインチューニング前後の平均スコアを比較し、出力例をレビューすることで、モデルが私たちの好みにどの程度合致するように改善されたかを確認できます。

In [ ]:
# generate responses
job = sync_client.fine_tuning.jobs.retrieve(ft.id)
if job.status == "succeeded":
    responses = await generate_responses(testset, model=job.fine_tuned_model)

    post_run = sync_client.evals.runs.create(
        name=ft.id,
        eval_id=logs_eval.id,
        data_source={
            "type": "responses",
            "source": {"type": "responses", "limit": len(test_pairs)},
        },
    )

In [59]:
# get scores from the evaluation
post_data = sync_client.evals.runs.output_items.list(
    eval_id=logs_eval.id, run_id=post_run.id
).data
post_scores = [s.results[0]["score"] for s in post_data]

# print scores & a sample comparison from the test set for illustration
print(
    "Δ mean:",
    sum(t - b for b, t in zip(base_scores, post_scores)) / len(base_scores),
)
print("\n=== SAMPLE COMPARISON ===")
idx = 0
print(f"Prompt:\n  {testset[idx]['item']['input']}\n")
print(f"Base model reply: \n {base_data[idx].sample.output[0].content} \n")
print(f"DPO-tuned model reply \n {post_data[idx].sample.output[0].content}")

Δ mean: 0.45

=== SAMPLE COMPARISON ===
Prompt:
  Can I upgrade to faster delivery if my package is still being processed?

Base model reply: 
 Whether you can upgrade to express shipping while your order is still being processed depends on the store's policies. Generally, many stores allow shipping upgrades before the order is shipped. 

To assist you better, could you please provide your order number or the name of the store you ordered from? Alternatively, you can contact the store's customer service directly to request the upgrade. 

DPO-tuned model reply 
 Hi! I’d be happy to help with that. If your package hasn’t shipped yet, there’s a good chance we can upgrade your delivery speed. Could you please provide me with your order number? I’ll check the status and let you know the available options for faster delivery.
